## Описание проекта


### Цель и условия проекта

Цель задачи:

Рассчитать достаточный объём сырья для безубыточной разработки новой скважины.
Выбрать скважины с максимальными значениями предсказаний. 
Просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям.
Рассчитать прибыль для полученного объёма сырья.
Найти среднюю прибыль, 95%-й доверительный интервал и риск убытков.
Предложить регион для разработки скважин.

Условия задачи:

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

### Критерии выбора

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

### Описание данных

Представленно 3 датасета, с одинаковыми названиями колонок и строк

`id` — уникальный идентификатор скважины;

`f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);

`product` — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

### Импорт библиотек и выгрузка данных

In [1]:
!pip install scikit-learn==1.1.3
!pip install imblearn
import pandas as pd
import numpy as np

from sklearn.preprocessing import  StandardScaler
from sklearn.linear_model import LinearRegression


from sklearn.metrics import  mean_squared_error
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 30.8 MB 47 kB/s  eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
     |████████████████████████████████| 226 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 31.5 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


In [2]:
try:
    data0 = pd.read_csv('/datasets/geo_data_0.csv')
except FileNotFoundError:
    data0 = pa.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
data0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
try:
    data1 = pd.read_csv('/datasets/geo_data_1.csv')
except FileNotFoundError:
    data1 = pa.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
data1.head()


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
try:
    data2 = pd.read_csv('/datasets/geo_data_2.csv')
except FileNotFoundError:
    data2 = pa.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
data2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


---данные выгрузились корректно

### Первичный осмотр данных

In [5]:
data0.info()
data1.info()
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

---в данных отсутсвуют пропуски, состоят из числовых и текстовых данных

In [6]:
data0.duplicated().value_counts() #явных дупликатов нет, все данные заполненны
data0.shape

(100000, 5)

In [7]:
data1.duplicated().value_counts()#явных дупликатов нет, все данные заполненны
data1.shape

(100000, 5)

In [8]:
data2.duplicated().value_counts()#явных дупликатов нет, все данные заполненны
data2.shape

(100000, 5)

---явных дупликатов нет, выборки равны друг другу по размеру, состоят из 100тыс.строк и 5 колонок

In [9]:
print(data0.describe(), data1.describe(), data2.describe(), sep='\n\n')

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%      

In [10]:
print(data0.corr(), data1.corr(), data2.corr(), sep='\n\n')

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000

               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000

               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


---данные в целом рабочие, присутсвуют выбросы и нулевые значения. но т.к. мы не знаем, что это за данные, то и бороться с аномалиями считаю некорректным

---присутствует очень сильная корреляция во втором датасете, в колонке f2 с целевым признаком. Мы не знаем что это такое, если бы мы знали что это такое,но мы не знаем что это такое, поэтому также считаю некорректным изменять эти данные

### Предобработка данных

In [11]:
def drop_id(data):
    return data.drop(['id'], axis=1)
data0 = drop_id(data0)
data1 = drop_id(data1)
data2 = drop_id(data2)

---колонка ID содержит в себе уникальные индификаторы скважин и не имеет статической значимости, поэтому считаю необходимым удалить эти колонки

### Выводы о данных

---данные выгружены корректно, состоят из численнных значений, не имеют пропусков и могут быть далее использованы в работе.

---была удалена колонка с индификаторами

--- данные в некоторой степени не равномерны и имеют разные показатели в разных колонках, но т.к. нам не сообщили, что это за показатели, то и бороться с неравномерностью считаю некорректным. 

--- необходимое примечание - далее данные из выборки data0 буду называть данными из региона 0 и, соответсвенно, с другими регионами - регион 1 и регион 2

## Обучение и проверка модели

### Разделение данных на обучающую и валидационную выборки в соотношении 75:25.

In [12]:
#pd.options.display.expand_frame_repr = False

def separation(data):  #функция по разбивке данных на трейн и валидацию
    target = data['product']
    target = target.reset_index(drop = True) # сразу скинем индексы, чтобы не было проблем в бустстрапировании
    features = data.drop(['product'], axis=1)
    return train_test_split(
        features, target, test_size=0.25, random_state=12345)

print(separation(data0)[0]) #вернулись разделенные данные с первого региона
print(separation(data1)[0]) #вернулись разделенные данные со второго региона
print(separation(data2)[0]) #вернулись разделенные данные с третьего региона

             f0        f1         f2
27212  0.022450  0.951034   2.197333
7866   1.766731  0.007835   6.436602
62041  0.724514  0.666063   1.840177
70185 -1.104181  0.255268   2.026156
82230 -0.635263  0.747990   6.643327
...         ...       ...        ...
4094   1.863680 -0.298123   1.621324
85412 -1.162682 -0.014822   6.819941
2177   0.862688 -0.403776   1.867662
77285  0.846235 -0.489533   1.058786
86498  2.019850  0.263887  11.497239

[75000 rows x 3 columns]
              f0         f1        f2
27212  -6.488552  -1.590478  3.001311
7866   18.819463   4.602079  2.996867
62041  10.816499  -3.919653  1.991077
70185 -12.416362  -9.343774  0.996691
82230 -15.041012  -8.474624  1.996463
...          ...        ...       ...
4094   -1.859774  -4.818321  2.998956
85412   4.863108  -6.071357  3.990522
2177   10.527491  -8.124357  5.004246
77285   0.110801 -10.896529  3.005078
86498  -4.653214  -4.283936  2.999043

[75000 rows x 3 columns]
             f0        f1        f2
27212 -0.906

---применил функцию для разделения на выборки в пропорции 75\25, в целевом признаке сбросил индексы, для корректной работы функции по расчету прибыли и вывел тренировочную часть для проверки корректности работы функции.

### Обучение моделей и  предсказания на валидационной выборке.

---применение стандартизации и модели линейной регрессии для первого региона

In [13]:
features_train, features_valid, target_train, target_valid = separation(data0)# разделение данных 

scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_valid = scaler.transform(features_valid)

model = LinearRegression()
model.fit(features_train,target_train) # обучение  модели на тренировочной выборке
predictions0 = model.predict(features_valid) # получение предсказания модели на валидационной выборке
predictions_mean_data0 = predictions0.mean()
result0 = mean_squared_error(target_valid, predictions0)**0.5
target_valid0 = target_valid.reset_index(drop=True)

---применение стандартизации и модели линейной регрессии для второго региона

In [14]:
features_train, features_valid, target_train, target_valid = separation(data1)

scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_valid = scaler.transform(features_valid)

model = LinearRegression()
model.fit(features_train,target_train) # обучение модели на тренировочной выборке
predictions1 = model.predict(features_valid) # получение предсказания модели на валидационной выборке
predictions_mean_data1 = predictions1.mean()
result1 = mean_squared_error(target_valid, predictions1)**0.5
target_valid1 = target_valid.reset_index(drop=True)

---применение стандартизации и модели линейной регрессии для третьего

In [15]:
features_train, features_valid, target_train, target_valid = separation(data2)

scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_valid = scaler.transform(features_valid)

model = LinearRegression()
model.fit(features_train,target_train) # обучение модель на тренировочной выборке
predictions2 = model.predict(features_valid) # получение предсказания модели на валидационной выборке
predictions_mean_data2 = predictions2.mean()
result2 = mean_squared_error(target_valid, predictions2)**0.5
target_valid2 = target_valid.reset_index(drop=True)


---предсказания модели и правильные ответы сохранены в ячейках с применениями моделей

### Средний запас предсказанного сырья и RMSE модели.

In [16]:
target_valid = target_valid0.mean(), target_valid1.mean(), target_valid2.mean()

result = result0, result1, result2
prediction_mean = predictions_mean_data0, predictions_mean_data1, predictions_mean_data2
means_err = pd.DataFrame({'ПРЕДСКАЗАННОЕ СРЕДНЕЕ': prediction_mean, 'RMSE': result, 'ЦЕЛЕВОЕ СРЕДНЕЕ': target_valid}, index = ['РЕГИОН 0', 'РЕГИОН 1', 'РЕГИОН 2'])
means_err

,ПРЕДСКАЗАННОЕ СРЕДНЕЕ,RMSE,ЦЕЛЕВОЕ СРЕДНЕЕ
РЕГИОН 0,92.592568,37.579422,92.078597
РЕГИОН 1,68.728547,0.893099,68.723136
РЕГИОН 2,94.965046,40.029709,94.884233


### Анализ результатов

--- в целом предсказанное и среднее по целевому признаку у каждого региона не сильно различаются, но т.к. подразумевается работа с бюджетом нескольких мегаполисов, то и суммы, скрывающиеся за долями процентов могут быть огромными.

---самая низкая средняя квадратичная ошибка у второго региона, но и доля целевого признака у него несколько меньше остальных. Можно было бы указать, что по предсказанию модели, мы можем выбрать регион 3, как самый перспективный. Но чтобы минимизировать риски, необходимо поработать с данными дальше

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраним в отдельных переменных.

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [17]:
n = 500 #число точек для разведки в регионе
n_top_200 = 200 #число точек для разработки в регионе
budget = 10**10 #бюджет для разработки в регионе
revenue_per_unit = 450 * 10**3 #доход с каждой единицы продукта(объем указан в тыс. баррелей)
losses_proba_limit = 2.5 #вероятность убытков должна быть меньше 2.5%

### Расчет достаточного объема сырья

достаточный объём сырья для безубыточной разработки новой скважины

In [18]:
sufficient_volume = budget / n_top_200 / revenue_per_unit

print("Достаточный объём сырья для безубыточной разработки новой скважины: {0:.0f} тыс. баррелей".format(sufficient_volume))

Достаточный объём сырья для безубыточной разработки новой скважины: 111 тыс. баррелей


In [19]:
#среднее по целевому признаку в каждом регионе
sufficient_volume0 = (round(data0["product"]).mean())
sufficient_volume1 =  (round(data1["product"]).mean())
sufficient_volume2 =  (round(data2["product"]).mean())

In [20]:
def sufficient_volumes(x, n):
    if x >= sufficient_volume:
        print('Достаточный объём сырья для безубыточной разработки в регионе', n, 'средние запасы = ', x)
    else:
        print('В регионе', n, 'недостаточный объём сырья для безубыточной разработки, средние запасы = ', x)
        
        
sufficient_volumes(sufficient_volume0, 0)
sufficient_volumes(sufficient_volume1, 1)
sufficient_volumes(sufficient_volume2, 2)

В регионе 0 недостаточный объём сырья для безубыточной разработки, средние запасы =  92.50062
В регионе 1 недостаточный объём сырья для безубыточной разработки, средние запасы =  68.85204
В регионе 2 недостаточный объём сырья для безубыточной разработки, средние запасы =  94.99858


### Выводы по этапу подготовки расчёта прибыли

--- по усредненным данным ни один из регионов не может быть прибыльным. Для безубыточной разработки необходим объем сырья более 110т.бар., в то время, как в каждом регионе менее 100т.бар.. Возможно, что для разработки можно было бы принять за основной регион 2, с максимальным показателем средних запасов - 95.т.бар. Но это среднее взято из выборки в 100т. скважин. 

--- но из статистики мы знаем, что среднее не значит истинное. поэтому необходимо уточнить, насколько мы уверенны в этом показателе

## Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [21]:
def revenue(target_valid, predicted_valid, count):
    target = pd.Series(target_valid,index=target_valid.index)#составление колонки из целевого признака
    probabilities = pd.Series(predicted_valid,index=target.index)
    probs_sorted = probabilities.sort_values(ascending=False)#сортировка предказаний модели
    selected = target.loc[probs_sorted.index][:count]#отбор 200 скважин
    return (revenue_per_unit * selected.sum())-budget#формула прибыли

---функция для расчета прибыли , выбора скважин по сортировке и по прибыли для полученного объема

### Выбор скважины с максимальными значениями предсказаний. 

In [22]:
revenue_0 = revenue(target_valid0, predictions0, 200)

In [23]:
revenue_1 = revenue(target_valid1, predictions1, 200)

In [24]:
revenue_2 = revenue(target_valid2, predictions2, 200)

### Рассчет прибыли для полученного объёма сырья.

In [25]:
result = revenue_0, revenue_1, revenue_2
revenue_all = pd.DataFrame({'ПРЕДСКАЗАННЫЕ ЗНАЧЕНИЯ ПРИБЫЛИ': result, }, index = ['РЕГИОН 0', 'РЕГИОН 1', 'РЕГИОН 2'])
revenue_all

,ПРЕДСКАЗАННЫЕ ЗНАЧЕНИЯ ПРИБЫЛИ
РЕГИОН 0,3.320826e+09
РЕГИОН 1,2.415087e+09
РЕГИОН 2,2.710350e+09


---в предсказанной прибыли, также, как и в средних показателях залежей, можно увидеть, что регион 0 и регион 2 имеют болший показатель и должны быть рекомендованны, как наиболее перспективные

## Расчёт прибыли и рисков 

---на этом этапе, чтобы минимизировать ошибку, мы применим технику бутсрепа - создадим тысячу случайных выборок для усреднения данных. согласно ЦПТ - чем больше данных, тем истиннее среднее

---далее мы выберем доверительный  интервал и посчитаем риск убытка - насколько сильно данные могут нас увести в минус

### Примение техники Bootstrap с 1000 выборок, чтобы найти распределение прибыли

In [26]:
state = np.random.RandomState(12345)

values = []
for i in range(1000):
    target_sub = target_valid0.reset_index(drop = True).sample(n = 500, replace = True, random_state = state)
    valid_sub = predictions0[target_sub.index]
    values.append(revenue(target_sub, valid_sub, 200))

values = pd.Series(values)
    
lower0 = values.quantile(0.025)
upper0 = values.quantile(0.975)
mean0 = values.mean()
risk0 = ((values < 0).mean() * 100, '%')

print(f'{mean0:,.2f}')
print(f'{lower0:,.2f}')
print(f'{upper0:,.2f}')

print(risk0)

425,938,526.91
-102,090,094.84
947,976,353.36
(6.0, '%')


In [27]:
state = np.random.RandomState(12345)

values = []
for i in range(1000):
    target_sub = target_valid1.reset_index(drop = True).sample(n = 500, replace = True, random_state = state)
    valid_sub = predictions1[target_sub.index]
    values.append(revenue(target_sub, valid_sub, 200))

values = pd.Series(values)
    
lower1 = values.quantile(0.025)
upper1 = values.quantile(0.975)
mean1 = values.mean()
risk1 = ((values < 0).mean() * 100, '%')
print(f'{mean1:,.2f}')
print(f'{lower1:,.2f}')
print(f'{upper1:,.2f}')

print(risk1)

515,222,773.44
68,873,225.37
931,547,591.26
(1.0, '%')


In [28]:
state = np.random.RandomState(12345)

values = []
for i in range(1000):
    target_sub = target_valid2.reset_index(drop = True).sample(n = 500, replace = True, random_state = state)
    valid_sub = predictions2[target_sub.index]
    values.append(revenue(target_sub, valid_sub, 200))

values = pd.Series(values)
    
lower2 = values.quantile(0.025)
upper2 = values.quantile(0.975)
mean2 = values.mean()
risk2 = (values < 0).mean() * 100, '%'
print(f'{mean2:,.2f}')
print(f'{lower2:,.2f}')
print(f'{upper2:,.2f}')

print(risk2)

435,008,362.78
-128,880,547.33
969,706,954.18
(6.4, '%')


In [29]:
mean_all = mean0, mean1, mean2
lower_all = lower0, lower1, lower2
upper_all = upper0, upper1, upper2
risk_all = risk0, risk1, risk2
means_err = pd.DataFrame({'ВЕРХНИЙ ПРЕДЕЛ': upper_all, 'НИЖНИЙ ПРЕДЕЛ': lower_all, 'СРЕДНЕЕ': mean_all,
                         'РИСК УБЫТКА:': risk_all}, index = ['Region0', 'Region1', 'Region2'])
means_err

,ВЕРХНИЙ ПРЕДЕЛ,НИЖНИЙ ПРЕДЕЛ,СРЕДНЕЕ,РИСК УБЫТКА:
Region0,9.479764e+08,-1.020901e+08,4.259385e+08,"(6.0, %)"
Region1,9.315476e+08,6.887323e+07,5.152228e+08,"(1.0, %)"
Region2,9.697070e+08,-1.288805e+08,4.350084e+08,"(6.4, %)"


---для регионов посчитанна прибыль и найдено распределение прибыли

### Выводы: предложение региона для разработки скважин

--- Выгруженные данные были изучены и обработаны, некоторые аномалии изучены - выбросы, корреляция, но не обработаны, т.к. данные не расшифрованы

--- После обработки и разделения данных модель линейной регрессии обучена для каждого отдельного региона и сохраненны предсказания в отдельные переменные

--- Дальнейший анализ показал, что регион 2 имеет высокий показатель по средним залежам сырья

--- Регион 0 имел высокий показатель по предсказанной прибыли 

--- Но после имитации правильного  распределния прибыли в указанных регионах был найден риск убытка не ниже 6%, в то время, как в регионе 1, несмотря на не высокие показатели средних запасов и невысокой предсказанной прибыли, риск убытка составляет 1 % и доверительный интервал самый высокий из предложенных

--- На основании предидущего пункта считаю возможным предложить регион 1(скачанный data1) для разработки скважин, как наиболее приыбльный и максимально лишенный риска убытка